# RAG 

#### Libraries

In [34]:
# processing:
import os
import re
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from sklearn.metrics.pairwise import cosine_similarity

# neo4j:
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector

# llm:
from langchain.chat_models.gigachat import GigaChat
from langchain_community.embeddings import GigaChatEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# retrieval:
from langchain.chains import RetrievalQAWithSourcesChain

# telegram:
import telebot
from time import sleep

#### .env

In [35]:
load_dotenv()

True

In [36]:
# neo4j:
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE= os.getenv('NEO4J_DATABASE')

# llm:
GIGACHAT_SCOPE = os.getenv('SCOPE')
GIGACHAT_AUTH = os.getenv('AUTH_DATA')

# telegram:
BOT_TOKEN = os.getenv('BOT_TOKEN')

#### Reference

In [37]:
reference = pd.read_excel('../docs/reference.xlsx')

## Setting Up Database

### Connection

In [38]:
embeddings = GigaChatEmbeddings(credentials = GIGACHAT_AUTH, verify_ssl_certs = False)

In [39]:
vectorestore = Neo4jVector.from_existing_index(embeddings,
                                          url = NEO4J_URI,
                                          username = NEO4J_USERNAME,
                                          password = NEO4J_PASSWORD,
                                          node_label = "Bullet",
                                          index_name = "bullet",
                                          keyword_index_name = "word",
                                          search_type = 'hybrid',
                                         )

In [40]:
graph = Neo4jGraph(url = NEO4J_URI, username = NEO4J_USERNAME, password = NEO4J_PASSWORD, database = NEO4J_DATABASE)

### Populate

In [41]:
import pickle

with open('collections.pkl', 'rb') as f:
    collections = pickle.load(f)

> **NOTE:** DON'T RUN!

In [42]:
# vectorestore = Neo4jVector.from_documents(collections,
#                                           embeddings,
#                                           url = NEO4J_URI,
#                                           username = NEO4J_USERNAME,
#                                           password = NEO4J_PASSWORD,
#                                           node_label = "Bullet",
#                                           index_name = "bullet",
#                                           keyword_index_name = "word",
#                                           text_node_property = "text",
#                                           embedding_node_property = "embedding",
#                                           search_type = 'hybrid',
#                                           create_id_index = True,
#                                          )

## Setting Up LLM


In [43]:
llm = GigaChat(model = "GigaChat-Pro", # "GigaChat-Pro" / "GigaChat-Plus" (32k)
               temperature = 0.3,
               top_p = 0.2,
               n = 1,
               repetition_penalty = 1,
               credentials = GIGACHAT_AUTH, 
               verify_ssl_certs = False)

In [44]:
parser = StrOutputParser()

## Prompt Engineering

#### Prompt Template

In [91]:
template = """
            Задача: Предоставлять детализированные ответы, используя указанные документы.

            Цитируйте конкретные пункты и статьи документов при ответе на вопросы, связанные с правилами.
            Объясните применение документов. Не повторяйте информацию и формулируйте ответы грамотно.

            Запрос: "{question}"
            Документы: "{context}"
            
            Ответ: Ответ должен содержать не более 200 слов и включать финальный вывод.
            """

prompt = ChatPromptTemplate.from_template(template)

In [61]:
llm.get_num_tokens(template)

100

#### Question Extraction

##### 3 Versions

In [92]:
def extract_question_one(text):
    
    chain = llm | parser

    template = f"""
            Задача: Идентифицировать вопросы в предоставленном тексте.

            Текст запроса: "{text}"
            
            Анализируйте текст и выделяйте явные вопросы. Если в тексте нет вопросов, сформулируйте их на основе имеющейся информации.
            Результаты представьте как список вопросов, разделенных запятыми. Стремитесь к ясности и краткости.
                
            Ответ должен быть конкретным и содержать все ключевые аспекты, без дополнительных комментариев.
            """
    
    return chain.invoke(template).replace('/', ' ')

In [210]:
def extract_question_two(text):
    
    chain = llm | parser

    template = f"""
            Задание: Определите и сформируйте вопросы, содержащиеся в тексте.

            Исходный текст: "{text}"
            
            Проанализируйте текст и выделите вопросы, присутствующие в тексте. 
            Если в тексте вопросов не обнаружено, на основе содержания текста сформируйте вопросы, 
            которые полностью самодостаточны и не требуют дополнительного контекста для понимания.
            Вопросы должны быть представлены в виде списка, каждый вопрос отделен запятой. Обеспечьте, 
            чтобы каждый вопрос был ясным, полным и самодостаточным.
                
            Результаты должны быть точными, без лишних уточнений или комментариев.
            """
    
    return chain.invoke(template).replace('/', ' ')

In [257]:
def extract_question_three(text):
    
    chain = llm | parser

    template = f"""
            Задача: Определить до пяти наиболее важных вопросов на основе анализа текста.

            Предоставленный текст: "{text}"
            
            Изучите текст и определите основные темы и идеи. Сформулируйте до пяти вопросов, каждый из которых должен отражать ключевые аспекты и детали текста. 
            Вопросы должны быть самостоятельными, т.е. понятными без ссылок на исходный текст. 
            Если текст изначально не содержит явных вопросов, разработайте их на основе наиболее существенных моментов информации.
            Каждый вопрос должен быть кратким и конкретным, и список не должен содержать более пяти вопросов.
                
            Ожидается, что выданные вопросы будут полны и ясны, позволяя получить глубокое понимание основных идей текста без возвращения к нему.
            """
    
    return chain.invoke(template).replace('/', ' ')


### Utilities

In [209]:
def find_relevant_bullets(inquiry):
    match = re.search(r'(\d+\.\d+)(?:\.\d+)?', inquiry)
    
    if match:
        number = match.group(1)
        
        cypher = f"""
                    MATCH (n:Document)
                    WHERE n.bullet STARTS WITH '{number}' AND n.bullet IS NOT NULL
                    RETURN n.text
                    """
        mentioned_bullets = graph.query(cypher)
        
        if mentioned_bullets:
            texts = [bullet['n.text'] for bullet in mentioned_bullets]
            
            all_embeddings = [embeddings.embed_query(text) for text in texts] + [embeddings.embed_query(inquiry)]
            inquiry_embedding = all_embeddings[-1]
            bullet_embeddings = all_embeddings[:-1]
            
            similarities = cosine_similarity([inquiry_embedding], bullet_embeddings)[0]
            top_indices = np.argsort(similarities)[::-1][:3]
            top_texts = [texts[i] for i in top_indices]
            
            return top_texts
        else:
            return "Отсутствуют"
    else:
        return "Отсутствуют"

## LLM Retrival Chain

#### Chain

In [252]:
retriever = vectorestore.as_retriever(search_kwargs={"k": 10, 'score_threshold': 0.95, 'lambda_mult': 0.25})

In [253]:
chain = RetrievalQAWithSourcesChain.from_chain_type(llm, # pro / plus
                                                    chain_type = "stuff", # "stuff" / "map_rerank" / "refine"
                                                    retriever = retriever,
                                                    return_source_documents = True,
                                                    reduce_k_below_max_tokens = True,
                                                    max_tokens_limit = 8000, # up to 32k with plus model
                                                    chain_type_kwargs = {"verbose": False,
                                                                         "prompt": prompt,
                                                                         "document_variable_name": "context"})

##### Outputs

In [265]:
letter = 3

inquiry = str(reference['letter'][letter-1].replace('\n', ' '))

question_one = extract_question_one(inquiry)
question_two = extract_question_two(inquiry)
question_three = extract_question_three(inquiry)

response = chain.invoke({"question": extract_question_one(inquiry)}, return_only_outputs = False)

print(f"Qestion 1:\n{question_one}\n")
print(f"Qestion 2:\n{question_two}\n")
print(f"Qestion 3:\n{question_three}\n")
# response['answer']

Qestion 1:
1. Может ли быть заключен договор с участником, который подал заявку на конкурентный отбор, полностью соответствующую техническому заданию и не отклоненную службой безопасности?
2. Какие возможны шаги и процедуры в данной ситуации?

Qestion 2:
1. Может ли быть заключен договор с участником, который подал заявку на конкурентный отбор, полностью соответствующую техническому заданию и не отклоненную службой безопасности, если только он один подал заявку?
2. Какие возможны шаги и процедуры в данной ситуации?
3. Какие рекомендации можно дать по данному вопросу?

Qestion 3:
1. Может ли быть заключен договор с участником, который подал заявку на конкурентный отбор и полностью соответствует техническому заданию, но был отклонен службой безопасности?
2. Какие шаги и процедуры следует предпринять в данной ситуации?
3. Какие возможны варианты действий в случае, когда только один участник подал заявку на конкурентный отбор?
4. Какие критерии должны быть учтены при принятии решения о зак

In [266]:
answer = response['answer']
answer

'1. Да, договор может быть заключен с участником, который подал заявку на конкурентный отбор, полностью соответствующую техническому заданию и не отклоненную службой безопасности.\n\n2. Возможные шаги и процедуры в данной ситуации включают:\n   - Проверку соответствия заявки участника требованиям технического задания и документации о конкурентном отборе.\n   - Анализ и оценку заявки участника на соответствие критериям отбора.\n   - Принятие решения о заключении договора с участником, если его заявка соответствует требованиям и критериям отбора.\n   - Подписание договора между участником и заказчиком на условиях, указанных в документации о конкурентном отборе и в заявке участника.\n   - Обеспечение исполнения договора, если это было установлено требованием документации о конкурентном отборе.\n\nОднако, необходимо учитывать, что в случае уклонения участника от заключения договора или непредставления обеспечения исполнения договора, заказчик может принять решение о признании запроса котир

In [54]:
source_documents = [doc.page_content for doc in response['source_documents']]
source_documents

['4. Основания для проведения закупки у единственного поставщика (подрядчика, исполнителя)',
 '1.1.2. На основании согласованного реестра закупок Инициатор в КЦ / Обществе Компания 1 осуществляет заключение договора с единственным поставщиком (подрядчиком, исполнителем).',
 '6.2.2.5. Закупка у единственного поставщика (подрядчика, исполнителя).',
 'обоснованием необходимости заключения договора с единственным поставщиком (подрядчиком, исполнителем). В качестве такого обоснования могут быть предусмотрены условия обращения товара, работы или услуги на товарных рынках, не позволяющие провести закупку иным способом, в том числе наличие ограниченной конкуренции, а также иные обстоятельства, которые свидетельствуют, что закупка у единственного поставщика (подрядчика, исполнителя) с позиций экономической эффективности предпочтительна для заказчика или по объективным причинам проведение ее в иной форме нецелесообразно.В этом случае до заключения договора Центральный орган управления закупками 

In [55]:
def get_bullets_or_source(response):
    bullets = []
    for doc in response['source_documents']:
        bullet = doc.metadata.get('bullet')
        if bullet is None:
            bullet = doc.metadata.get('source')
        bullets.append(bullet)
    return bullets

meta = get_bullets_or_source(response)
print(meta)

['3.1.', '1.1.2.', '6.2.2.5.', '4.1.25.', 'Закупка у ЕдП', 'ПоЗ', 'ПоЗ', 'Закупка у ЕдП', 'ПоЗ', 'Закупка у ЕдП']


In [56]:
mentioned_bullets = find_relevant_bullets(inquiry)
mentioned_bullets

'Отсутствуют'

## Test

#### Telegram

In [ ]:
user_conversations = {}
bot = telebot.TeleBot(BOT_TOKEN)

In [ ]:
@bot.message_handler(content_types=['audio', 'video', 'document', 'photo', 'sticker', 'voice', 'location', 'contact'])
def not_text(message):
    user_id = message.chat.id
    bot.send_message(user_id, 'Я работаю только с текстовыми сообщениями!')

@bot.message_handler(content_types=['text'])

def handle_text_message(message):
    user_id = message.chat.id
    extracted_question = extract_question_one(message.text)

    response = chain.invoke({"question": extracted_question}, return_only_outputs = False)

    bot.send_message(user_id, str(response['answer']))
    sleep(2)

In [ ]:
bot.polling(none_stop=True)

#### Responses

In [260]:
responses = pd.DataFrame(columns = ['letter', 'extracted questions', 'answer', 'rag answer', 'sources', 'mentioned bullets'])

for letter in range(1, 21):

    inquiry = str(reference['letter'][letter-1].replace('\n', ' ')) # letters from excel
    refer = str(reference['answer '][letter-1]).replace('\n', ' ') # reference answer from excel

    question = extract_question_three(inquiry)
    response = chain.invoke({"question": question}, return_only_outputs = False)
    rag_answer = response['answer']
    sources = get_bullets_or_source(response)
    mentioned_bullets= str(find_relevant_bullets(inquiry)).replace('/', ' ')

    responses = pd.concat([responses, pd.DataFrame({'letter': [inquiry], 
                                                    'extracted questions': [question], 
                                                    'answer': [refer], 
                                                    'rag answer': [rag_answer], 
                                                    'sources': [sources],
                                                    'mentioned bullets': [mentioned_bullets]})
                                                    ], ignore_index = True)

In [261]:
responses.head()

,letter,extracted questions,answer,rag answer,sources,mentioned bullets
0,"Уважаемые коллеги, Добрый день! У меня возник...",1. Какие методы допустимы при расчете начально...,"Согласно пункту 5.11 ПоЗ, при закупках товаров...",Для расчета начальной максимальной цены (НМЦ) ...,"[ПоЗ, ПоЗ, 4.5.13., 5.11., ПоЗ, ПоЗ, 4.5.10., ...",Отсутствуют
1,"Уважаемые коллеги, Обращаюсь с вопросом: При ...",1. Кто должен согласовать диапазон цен при аре...,Согласно СК-03.03.02.01 Проведение закупки у е...,Для согласования диапазона цен при аренде АЗС ...,"[3.2., Закупка у ЕдП Приложение 2, Закупка у Е...",Отсутствуют
2,"Уважаемые коллеги, У меня вопрос, связанный с...",1. Может ли быть заключен договор с участником...,"пункт 17.1.8. в случае, когда на участие в кон...","1. Участник, который подал заявку на конкурент...","[ПоЗ, ПоЗ, ПоЗ, ПоЗ, ПоЗ, ПоЗ, ПоЗ, ПоЗ, ПоЗ, ...",Отсутствуют
3,"Уважаемые участники, Обращаюсь с вопросом, ка...",1. Какие требования и правила применяются к ко...,"Согласно пункту 1.2.43 ПоЗ, участником закупки...","Для коллективных участников, включая физически...","[ПоЗ, ПоЗ, ПоЗ, ПоЗ, ПоЗ, ПоЗ, ПоЗ, ПоЗ, ПоЗ, ...",Отсутствуют
4,"Добрый день! Подскажите, пожалуйста, по дате з...",1. Какая дата заключения договора ВЗЛ является...,Датой подведения итогов закупки для способа за...,"Для ответа на вопросы, связанные с датами закл...","[Формирование ГПЗ, 5.6.2., Формирование ГПЗ, Ф...",Отсутствуют


In [262]:
responses.to_excel('rag_test_three.xlsx', index=False)

## Compute Score

In [ ]:
from FlagEmbedding import BGEM3FlagModel

cross_model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=False)

test = pd.read_excel('explore/rag_test_three.xlsx') # read xlsx test file

def cross_score(model_answer, ans):
    score = cross_model.compute_score([ans, model_answer], max_passage_length=128, weights_for_different_modes=[1.0, 1.0, 1.0])['colbert']
    return score

results = pd.DataFrame(columns=['model_answer', 'reference_answer', 'score'])

for letter in range(1, 21):
    model_answer = str(test['rag answer'][letter-1].replace('\n', ' ')) # letters from xlsx
    ans = str(test['answer'][letter-1]).replace('\n', ' ') # reference answer from xlsx

    score = cross_score(model_answer, ans)


    current_result = pd.DataFrame({
        'model_answer': [model_answer],
        'reference_answer': [ans],
        'score': [score]
    })

    results = pd.concat([results, current_result], ignore_index=True)

results.to_excel('comparison_results.xlsx', index=False)

In [ ]:
test = pd.read_excel('comparison_results.xlsx')

In [ ]:
test['score'][:18]

In [ ]:
test['score'][:18].mean()

#### Basic Queries

[Cypher](https://neo4j.com/docs/cypher-cheat-sheet/5/auradb-enterprise)

In [ ]:
print(graph.schema)

Node properties are the following:
Chunk {embedding: LIST, id: STRING, text: STRING, source: STRING, bullet: STRING},Document {embedding: LIST, id: STRING, text: STRING, source: STRING, bullet: STRING},Test {embedding: LIST, id: STRING, text: STRING, source: STRING, bullet: STRING}
Relationship properties are the following:

The relationships are the following:



In [ ]:
cypher = """
         SHOW VECTOR INDEXES
         """
graph.query(cypher)

In [207]:
cypher = """
         MATCH (n)
         RETURN count(n)
         """
graph.query(cypher)

[{'count(n)': 3982}]

In [208]:
cypher = """
         MATCH (n:Bullet {bullet: "2.1."})
         RETURN n.text AS text
         """
graph.query(cypher)

[{'text': '2.1. Планирование закупок ПАО «Компания 1» и Обществ Компания 1 осуществляется путем составления годового плана закупок ПАО «Компания 1» и Обществ Компания 1 на календарный год (далее также – Годовой план закупок), а также планов закупок Заказчиков. Годовой план закупок является основанием для осуществления закупок. Перечень конкурентных и неконкурентных закупок определенного Заказчика, включенных в Годовой план закупок, включается Заказчиком в состав плана закупок Заказчика.'},
 {'text': '2.1. В настоящем стандарте содержатся ссылки на следующие нормативные и организационно-распорядительные документы Группы компаний ГПН: КТ-004 «Термины и сокращения» ; ПК-00 «Управление системой стандартизации»; М-13.07.11-01 «Методика определения лимита авансового платежа контрагентам при закупке товаров, работ, услуг»; Положение о закупках товаров, работ, услуг ПАО\xa0«Компания 1», утвержденное решением Совета директоров ПАО\xa0«Компания 1» 29\xa0марта 2019 года, протокол №\xa0ПТ-0102/14;

In [219]:
def query_bullet(number):
    if isinstance(number, str) and number.replace('.', '').isdigit():
        number = number.rstrip('.').strip() + '.'
    else:
        return "Invalid input. Please enter a bullet number in the format 'number.'"
    
    cypher = f"""
                MATCH (n:Bullet)
                WHERE n.bullet = '{number}' AND n.bullet IS NOT NULL
                RETURN n.text
                """
    try:
        result = graph.query(cypher)
        if result:
            return result
        else:
            return "No data found for the specified bullet number."
    except Exception as e:
        return f"An error occurred: {str(e)}"

In [232]:
bullet_number = '1.2.'
query_bullet(bullet_number)

[{'n.text': '1.2. Термины и определения'},
 {'n.text': '1.2. При распространении НМД через механизм тиражирование:'},
 {'n.text': '1.2. При проведении закупок для оказания услуг по авторскому контролю за разработкой проектной документации при НМЦ договора 10 млн руб. без НДС и более Инициатор также оформляет комплект документов в соответствии с шаблоном (Ш-03.03.02.01-04).'}]